<h2> Generate Acceptance Test Python code covering open api specification behaviour using ibm-granite-code hosted on Replicate</h2>

<h3>Notebook goals:</h3>

The learning goals of this notebook are:

1. Connect to `ibm-granite/granite-8b-code-instruct-128k` hosted on Replicate, and use it to Generate code for Acceptance Test Python Code
2. Connect to `ibm-granite/granite-20b-code-instruct-8k` hosted on Replicate, and use it to Generate code for Acceptance Test Python Code


<h3>Prerequisites: </h3>

1. Create an account on Replicate.
2. Copy the Replicate API Token to an environment file (.env file) and place it in the same directory as this notebook. Environment variable can be named as `REPLICATE_API_TOKEN`.
3. Install `langchain` and `ibm-granite-community/utils` python packages using below pip command.

In [ ]:
%pip install langchain
%pip install git+https://github.com/ibm-granite-community/utils

## Introduction


<h3>Acceptance tests: OpenAPI specification</h3>

1. Designed for user, who inputs (chat UI) or selects (IDE plugin) an OpenAPI specification to request generation of acceptance tests for it.
2. Creates a response from the system that returns the acceptance test Python code that covers the behavior of the API.


## Using a local model

<h3>Get the variables from .env file</h3>

This guide will demonstrate a basic inference call using the replicate package.

To establish an authenticated session, the code below imports necessary packages, and loads the environment variable `REPLICATE_API_TOKEN` from the .env file.

In [ ]:
import os
from dotenv import load_dotenv
from ibm_granite_community.langchain_utils import find_langchain_model

# Load environment variables from the .env file
load_dotenv()

# Retrieve the API token directly from environment variables
replicate_api_token = os.getenv('REPLICATE_API_TOKEN')

if replicate_api_token:
    print('API token loaded successfully:', replicate_api_token)
else:
    print('Failed to load API token. Please check your .env file.')

## Using a remotely-hosted model

The Granite Code models are available on [Replicate](https://replicate.com/).

At the moment, they are only available to members of the Granite Code team.
Request an invite to get access.

This guide will demonstrate a basic inference call using the `replicate` package as well
as via LangChain.
In both cases, you will provide a [Replicate API Token](https://replicate.com/account/api-tokens).


<h3>Deployment ID and Model ID Specifications</h3>

Mention the Deployment ID as granite and Specify the Model ID to be used for building the use-case.

In [18]:
deployment_id = "ibm/granite-dev"

model_id = "ibm/granite-8b-code-instruct:50da94a0b1b5d28e3161d1312077d856eb673b87e633438362e4820fed563444"

<h3>Prompt Definition</h3>

We further need to specify the prompt as an input to the model with apt configuration by including within it the specification along with the correct requirements needed. 

For an instance, in this use case, we implement the prompt to contain an instruction for generating acceptance test in the form of python code to cover the behavior of application of open api onto testing orders.

In [19]:
prompt = "Please generate acceptance test Python code that covers the behavior of the open api specification as per the specification \\n\\nopenapi: 3.0.0\\ninfo:\\n  title: Test Generation System\\n  description: API for creating test scenarios for orders\\n  version: 1.0.0\\npaths:\\n  /orders/count:\\n    get:\\n      summary: Test the fetch query for the count of orders in the system\\n      responses:\\n        200:\\n          description: OK\\n          content:\\n            application/json:\\n              schema:\\n\""

### Replicate package

Python package replicate needs to be installed for being imported in the next step.

In [ ]:
%pip install replicate

<h3>Import all the necessary libraries and packages</h3>

Replicate package needs to be imported in order to be used for refering to the deployment for which the prompt is being directed to achieving the test/s generated.

In [21]:
import replicate

<h3>Fetching Acceptance Test as response to the prompt</h3>

Next, using replicate, prediction is created with the prompt designed as per the configuration in the previous step. Furthermore, the exception handling is curated onto the response of the prompt to generate clean output for the user in terms of the test generated inclusing error handling.

In [ ]:
deployment = replicate.deployments.get(deployment_id)
print("prompt", prompt)
prediction = deployment.predictions.create(
  input={"prompt": prompt}
)

prediction.wait()

if (prediction.output[0].__contains__('mock')): 
  print('The provided input specification seems corrupted. Kindly check again and reattampt.')
else:
  print(prediction.output[0].split('```python')[1])
